**Chapter 10 - Creating Special Features in Your PDF**

There are several special features that you can utilize with ReportLab
when generating a PDF. We will be covering the following topics in this
chapter:
- File Annotations
- Bookmarks
- Page Transitions
- Encryption
- Interactive Forms

**File Annotations**

You can add metadata to your PDF with ReportLab as well, though these annotations only seem to show up in Adobe Reader. This is
accomplished by calling the following self-explanatory canvas methods:
- `setAuthor()`
- `setTitle()`
- `setSubject()`

File annotations are invisible. You can only view them by opening up the
`Document Properties` dialog, which can be found in the *File –> Properties* menu item in Adobe Reader. This is a great way to provide
simple standard information about the document to archiving software or
other software that might need to consume your PDFs.

In [1]:
# annotations.py

from reportlab.pdfgen import canvas

def annotations():
    my_canvas = canvas.Canvas('annotations.pdf')

    my_canvas.drawString(10, 700, 'Annotations demo')

    my_canvas.setAuthor('Mike Driscoll')
    my_canvas.setTitle('ReportLab: PDF Processing with Python')
    my_canvas.setSubject('Use Python to create PDFs')

    my_canvas.save()

if __name__ == '__main__':
    annotations()

**Bookmarks**

ReportLab has support for adding Bookmarks to your PDFs. A bookmark is
an internal hyperlink that you can click on (in the Adobe Reader sidebar, for example) to navigate the document.
ReportLab just supports the ability to jump from one part of the
document to another part. You can also control the zoom level in the
document reader after the jump. To add a bookmark to your document, you
will need to use the `bookmarkPage` canvas method. To make the
bookmark visible to be clicked on, you will also need to call the
`addOutlineEntry` method.
The `bookmarkPage` canvas method signature looks like this:
```python
canvas.bookmarkPage(name,
                    fit="Fit",
                    left=None,
                    top=None,
                    bottom=None,
                    right=None,
                    zoom
                   )
```

The default of the `bookmarkPage` canvas method is for it to define
the page itself as the destination to jump to. It will also scale the
entire page to fit the screen by default.

Here is a table that explains the various ways that the fit argument
can be set:

 Fit  | Parameters Required   | Description                                                                               
------|-----------------------|------------------------------------------------------------------------------
 Fit  |                       | The entire page will fit in the window (default)                                          
 FitH | top                   | The top coord at the top of the window with the width scaled to fit                       
 FitV | left                  | The left coord at the left of the window with the height scaled to fit                    
 FitR | left bottom right top | Scales the window to fit the specified rectangle                                          
 XYZ  | left top zoom         | Fine grained control. If a parameter is omitted, the PDF interprets that as “leave as is” 

The *Parameters Required* column describes which of the other
parameters for the `bookmarkPage` will need to be set for that version
of the `fit` argument to work correctly.

In [2]:
# bookmark_demo.py

from reportlab.pdfgen import canvas

def bookmark_demo():
    my_canvas = canvas.Canvas('bookmarks.pdf')

    my_canvas.drawString(10, 700, 'Page 1')
    my_canvas.bookmarkPage('page1')                 #add bookmark to the current page

    my_canvas.showPage()                            #creates page break so that any other drawing calls to canvas will be made on next page

    my_canvas.drawString(10, 700, 'Page 2')
    my_canvas.bookmarkPage('page2')

    my_canvas.addOutlineEntry('Page 1', 'page1')    #name that will be visible, and which bookmark 'object' we're linking to
    my_canvas.addOutlineEntry('Page 2', 'page2')    #syntax: (title, key)
    
    my_canvas.save()

if __name__ == '__main__':
    bookmark_demo()

**The Outline Tree**

ReportLab calls the navigation sidebar an *Outline Tree*. When you
want to add an outline entry to the outline tree, you need to use the
previously mentioned `addOutlineEntry` method. A lot of users like
navigating their documents using the sidebar, so knowing how to add
entries to that sidebar can be valuable.
As we learned just a bit ago, there are four arguments that you can pass
to the `addOutlineEntry` method: `title`, `key`, `level`, and `closed`. The
`title` argument is the caption that will appear in your navigation
pane. The `key` is a unique string within the document that you used
when creating a bookmark. The `level` argument defaults to zero, which
is the uppermost level. You may only go down one level at a time. If you
try to do go down more than one level, an error will be raised. Lastly,
the `closed` argument tells the navigation pane whether or not the
bookmark should be closed or opened.

**Encryption**

***NB:*** I have found ReportLab's encryption to be insufficient - regardless of the value of `canModify` and the presence/absence of user and owner passwords, I'm able to modify PDF contents. PyPDF2 may provide more functionality here.

----------------------------------------------------------------------------

Adobe’s implementation of the PDF specification allows you to encrypt
your PDFs. You may do any of the following things when encrypting your
PDFs:
- Password protect the document
- Encrypt the contents of the file
- Control the ability to print, copy and paste or edit the document in the viewer

Note that an encrypted PDF does not require a password, but when you do
password protect your PDF, it is encrypted automatically. When the user
enters their password, the PDF will open the document, decrypt it and
then display it for the user to read. There are two different types of
passwords:
- The “owner” password - Basically gives you administrator privileges
- The “user” password - Allows you to open the document for viewing

If you have the “owner” password, you can print, copy/paste and change
the password itself. When you use the “user” password, your options may
be restricted. The following describes what permissions you can set:
- Modification of the document
- Copying text / graphics
- Adding / modifying annotations or form fields
- Printing

If you set the user password to a blank string, then Adobe will not
prompt you for a password. If you only set the owner password, then the
PDF will open without a prompt. Finally, if you set the password for the
owner and the user to the same string, then the PDF will only open in
restricted mode with whatever privileges you have set. You will not be
able to modify any of those settings in this case. Always set the owner
and the user password to different strings.

When the PDF is encrypted, the encryption will apply to all the strings
and streams in the file. This will prevent nefarious people from trying
to find the password in a text or hex editor and changing it. According
to ReportLab’s documentation, PDF’s use an MD5 hash and the RC4
encryption algorithm for encrypting the PDF. RC4 is a symmetric stream
cipher that uses the same mechanic for encrypting and decrypting your
file without changing the length of the data.

Now we are ready to learn how to actually encrypt a PDF using ReportLab.
You will need to import `pdfencrypt` from `reportlab.pdfgen`. Then
to encrypt the document, you will create an instance of the
`StandardEncryption` class. Its `__init__()` looks like this:
```python
def __init__(self, userPassword,
             ownerPassword=None,
             canPrint=1,
             canModify=1,
             canCopy=1,
             canAnnotate=1,
             strength=40):
```

Everything is set to a default except for the `userPassword` argument.
The `ownerPassword` is just set to None. The next four arguments set
various privileges on the PDF document itself. The last is the
encryption strength. Looking at the source, it looks like the only valid
values are 40 and 128 with the default being the weakest security.

 Owner password set? | User password set? | Result                                                                                        
---------------------|--------------------|-----------------------------------------------------------------------------------------------
 Y                   | -                  | No password required when opening file. Restrictions apply to everyone.                       
 \-                   | Y                  | User password required when opening the file. Restrictions apply to everyone.                 
 Y                   | Y                  | A password required when opening the file. Restrictions apply only if user password supplied. 

In [15]:
# encryption_demo.py

from reportlab.lib import pdfencrypt
from reportlab.pdfgen import canvas

def encryption_demo(userPassword, ownerPassword,
                    canPrint=1, canModify=0, canCopy=1, canAnnotate=0):
    
    encrypt = pdfencrypt.StandardEncryption(
        userPassword=userPassword,
        ownerPassword=ownerPassword,
        canPrint=canPrint,
        canModify=canModify,
        canCopy=canCopy,
        canAnnotate=canAnnotate,
        strength=128)
    my_canvas = canvas.Canvas('encryption_demo.pdf', encrypt=encrypt)

    my_canvas.drawString(20, 750, 'This is page one')
    my_canvas.save()

if __name__ == '__main__':
    encryption_demo(userPassword='e',
                    ownerPassword='0wner_pwd')
    

**Interactive Forms**

ReportLab also allows you to create interactive fillable forms using
their toolkit. The PDF standard actually has a fairly rich set of
interactive elements. ReportLab doesn’t support all of these elements,
but it does cover most of them. In this section, we will look at the
following widgets:
- checkbox
- radio
- choice
- listbox
- textfield

All of these widgets are created by calling various methods on the
`canvas.acroform` property. Note that you can only have one form per
document. Let’s take a look at the widgets that ReportLab supports!

**Checkbox**

The `checkbox` widget is exactly what it sounds like. It’s a little
box that you can check. ReportLab supports several different “check”
styles though, so when the checkbox is checked, it can look different
depending on the style that you set. Here are all the parameters that
you can set for a checkbox:

Parameter       | Meaning                                                     | Default    
-----------------|-------------------------------------------------------------|------------
 name            | The parameter’s name                                        | None       
 x               | The horizontal absolute coordinate position                 | 0          
 y               | The vertical absolute coordinate position                   | 0          
 size            | The outline dimensions (size x size)                        | 20         
 checked         | If True, the checkbox is checked                            | False      
 buttonStyle     | The checkbox’s style (see below for more information)       | ‘check’    
 shape           | The outline of the widget (see below for more information)  | ‘square’   
 fillColor       | The color filling the widget                                | None       
 textColor       | The color of the text / symbol                              | None       
 borderWidth     | The width of the border                                     | 1          
 borderColor     | The border color                                            | None       
 borderStyle     | The border style name                                       | ‘solid’    
 tooltip         | The text to show when hovering over the widget              | None       
 annotationFlags | Space separated string of flags for annotation              | ‘print’    
 fieldFlags      | Space separated string of field flags                       | ‘required’ 
 forceBorder     | When True, a border is drawn                                | False      
 relative        | When True, obey the current canvas transform setting        | False      
 dashLen         | The dashline to be used if the borderStyle==’dashed’        |            
